In [8]:
from importlib import reload
import numpy as np
import helperfuncs as hp
import ldsc_72 as ld
import matplotlib.pyplot as plt
import scipy.stats
reload(ld)

<module 'ldsc_72' from 'C:\\Users\\Hariharan\\Documents\\git_repos\\SNIPar\\ldsc_reg\\infertheta\\ldsc_72.py'>

In [9]:
# Getting model set up

np.random.seed(123)
N = 100
S_size=  int(N/2)
S = np.array([100/N] * N).reshape((N, 1, 1))
V = np.identity(1) * 100

model = ld.sibreg(S = S)
model.simdata(V, N, simr = True)

Warning there is no value for theta. Maybe consider simulating it
No value for U given. Generating a vector of ones (all SNPs weighted equally)
No value for r given. Generating a vector of ones for r
Simulated LD scores!
Effect Vectors Simulated!


In [10]:
negll, neggrad = model.neg_logll_grad(V)

In [11]:
neggrad

array([-34.79564052])

In [12]:
model.solve(est_init = np.atleast_2d(0))

(array([[468.95261992]]),       fun: 274.6392748836826
  hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
       jac: array([-0.01183156])
   message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
      nfev: 54
       nit: 15
    status: 0
   success: True
         x: array([468.95261992]))

In [20]:
def logll(V):

    Vin = hp.extract_upper_triangle(V)
    logll = -model.neg_logll_grad(Vin)[0]
    
    return logll

def gradlogll(V):

    Vin = hp.extract_upper_triangle(V)
    grad = -model.neg_logll_grad(Vin)[1]
    
    return grad

def num_grad(V, row = 0, col = 0, dx = 1e-6):
    
    V1 = np.copy(V)
    V2 = np.copy(V)
    V1[row, col] += dx
    V2[row, col] -= dx
    
    derivative = (logll(V1) - logll(V2))/2 * dx
    
    return derivative

def jacobian(V, dx = 1e-6):
    
    rows = V.shape[0]
    cols = V.shape[1]
    
    J = np.empty((rows, cols))
    
    for row in range(rows):
        for col in range(cols):
            J[row, col] = num_grad(V, row = row, col = col)
    
    return J

num_grad(V)

9.877539355329645e-13

In [21]:
jacobian(V)

array([[9.87753936e-13]])

In [22]:
-model.neg_logll_grad(V)[1][0]

34.795640523907664

Comparing Log Likelihood Functions

In [23]:
# comparing log likelihoods

Si = S[0]
ri = model.r[0]
logll = scipy.stats.multivariate_normal.logpdf(model.theta, 
                                               mean = np.zeros(1),
                                               cov = Si + ri * V/N)
logll.sum()

-326.3238961757506

In [24]:
-model.neg_logll_grad(V)[0]

-343.2478882273632